In [42]:
import json

with open('reddit data/left.json') as f:
    left = json.load(f)
with open('reddit data/right.json') as f:
    right = json.load(f)
with open('reddit data/left_subs.json') as f:
    left_subs = json.load(f)
with open('reddit data/right_subs.json') as f:
    right_subs = json.load(f)
with open('reddit data/left_users.json') as f:
    left_users = json.load(f)
with open('reddit data/right_users.json') as f:
    right_users = json.load(f)
with open('reddit data/all_subs.json') as f:
    all_subs = json.load(f)
with open('reddit data/left_sources.txt') as f:
    left_sources = [l.strip().lower() for l in f.readlines() if l.strip() != '']
with open('reddit data/right_sources.txt') as f:
    right_sources = [l.strip().lower() for l in f.readlines() if l.strip() != '']


In [88]:
import matplotlib.pyplot as plt
import networkx as nx
from pprint import pprint

In [37]:
# invert user:subs dict
sub_users = dict()
for s in all_subs:
    sub_users[s] = set()
for u in left:
    for s in left[u]:
        sub_users[s].add(u)
for u in right:
    for s in right[u]:
        sub_users[s].add(u)

In [53]:
# filter out small subs
counts = {s:len(sub_users[s]) for s in sub_users}
large_subs = {s:sub_users[s] for s in sub_users if len(sub_users[s]) >= 1000}
print(len(large_subs))
large_sub_users = set()
for s in large_subs:
    for u in large_subs[s]:
        large_sub_users.add(u)
print(len(large_sub_users))

102
38893


In [119]:
large_sub_edges = dict()
sources_set = set(left_sources).union(set(right_sources))
sorted_subs = sorted(list(large_subs))
max_weight = 0
for i in range(len(sorted_subs)):
    if sorted_subs[i] not in sources_set: continue
    s = sorted_subs[i]
    if s not in large_subs: continue
    for u in large_sub_users:
        if u in large_subs[s]:
            for j in range(i + 1, len(sorted_subs)):
                s2 = sorted_subs[j]
                if u in large_subs[s2]:
                    count = large_sub_edges.get((s, s2), 0)
                    large_sub_edges[(s, s2)] = count + 1
                    if count + 1 > max_weight: max_weight = count + 1

pprint(large_sub_edges)

{('asktrumpsupporters', 'assholedesign'): 17,
 ('asktrumpsupporters', 'atheism'): 44,
 ('asktrumpsupporters', 'aww'): 83,
 ('asktrumpsupporters', 'baseball'): 41,
 ('asktrumpsupporters', 'beholdthemasterrace'): 33,
 ('asktrumpsupporters', 'bestof'): 91,
 ('asktrumpsupporters', 'blackpeopletwitter'): 98,
 ('asktrumpsupporters', 'bluemidterm2018'): 41,
 ('asktrumpsupporters', 'books'): 44,
 ('asktrumpsupporters', 'choosingbeggars'): 27,
 ('asktrumpsupporters', 'circleoftrust'): 50,
 ('asktrumpsupporters', 'conservative'): 82,
 ('asktrumpsupporters', 'conspiracy'): 105,
 ('asktrumpsupporters', 'crappydesign'): 16,
 ('asktrumpsupporters', 'cringeanarchy'): 57,
 ('asktrumpsupporters', 'damnthatsinteresting'): 21,
 ('asktrumpsupporters', 'dankmemes'): 38,
 ('asktrumpsupporters', 'dataisbeautiful'): 66,
 ('asktrumpsupporters', 'democrats'): 16,
 ('asktrumpsupporters', 'documentaries'): 33,
 ('asktrumpsupporters', 'enoughtrumpspam'): 28,
 ('asktrumpsupporters', 'esist'): 49,
 ('asktrumpsupport

 ('esist', 'nba'): 274,
 ('esist', 'news'): 1936,
 ('esist', 'nfl'): 238,
 ('esist', 'niceguys'): 235,
 ('esist', 'nostupidquestions'): 211,
 ('esist', 'nottheonion'): 457,
 ('esist', 'oddlysatisfying'): 280,
 ('esist', 'oldschoolcool'): 511,
 ('esist', 'pcmasterrace'): 222,
 ('esist', 'personalfinance'): 252,
 ('esist', 'pics'): 1482,
 ('esist', 'politicalhumor'): 1290,
 ('esist', 'politics'): 2874,
 ('esist', 'prequelmemes'): 201,
 ('esist', 'publicfreakout'): 216,
 ('esist', 'reactiongifs'): 299,
 ('esist', 'sandersforpresident'): 190,
 ('esist', 'science'): 514,
 ('esist', 'showerthoughts'): 824,
 ('esist', 'socialism'): 115,
 ('esist', 'space'): 281,
 ('esist', 'sports'): 245,
 ('esist', 'starterpacks'): 405,
 ('esist', 'starwars'): 317,
 ('esist', 'technology'): 775,
 ('esist', 'television'): 600,
 ('esist', 'the_donald'): 177,
 ('esist', 'the_mueller'): 821,
 ('esist', 'therewasanattempt'): 234,
 ('esist', 'tinder'): 210,
 ('esist', 'todayilearned'): 1514,
 ('esist', 'trashy'): 

In [124]:
G = nx.Graph()
for e in large_sub_edges:
    G.add_edge(e[0], e[1], weight=large_sub_edges[e] / max_weight * 20)
    
colors = dict()
left_sources_set = set(left_sources)
right_sources_set = set(right_sources)
for s in G.nodes:
    if s in left_sources_set:
        colors[s] = 'blue'
    elif s in right_sources_set:
        colors[s] = 'red'
    else:
        colors[s] = 'white'

In [81]:
import math

from bokeh.io import show, output_file, output_notebook, reset_output
from bokeh.plotting import figure
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, WheelZoomTool, PanTool
from bokeh.models import GraphRenderer, StaticLayoutProvider, Oval
from bokeh.palettes import Spectral8
from bokeh.models.graphs import from_networkx


In [126]:

size = 2
plot = figure(title='Graph Layout Demonstration', x_range=(-size, size), y_range=(-size, size),
              tools='')

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))

### start of layout code

hover = HoverTool(tooltips=[("sub name", "@name")])
hover.show_arrow = False
plot.add_tools(hover, BoxZoomTool(), PanTool(), WheelZoomTool())
graph.node_renderer.data_source.data['name'] = list(G.nodes)

# graph_layout = dict(zip(G.nodes, zip(x, y)))
# graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

graph.edge_renderer.data_source.data["line_width"] = [G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
graph.node_renderer.data_source.data['node_color'] = [colors[n] for n in G.nodes]
graph.node_renderer.glyph = Circle(size=10, fill_color={'field': 'node_color'})
graph.edge_renderer.glyph.line_width = {'field': 'line_width'}
plot.renderers.append(graph)
reset_output()
output_notebook()
show(plot)

Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: line_width [renderer: GlyphRenderer(id='b4904ec0-4249-46ee-822f-ed8a4ce317de', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: node_color [renderer: GlyphRenderer(id='d632705b-8855-485a-8d4a-234e6434c245', ...)]
